In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [4]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [5]:
dataset["classification_yes"].value_counts()

1    249
0    150
Name: classification_yes, dtype: int64

In [6]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes'  ]]
dep=dataset[['classification_yes' ]]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
x_train=sc.fit_transform(X_train)
x_test=sc.transform(X_test)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
param_grid = {'criterion':['gini', 'entropy', 'log_loss'],'max_features':['sqrt','log2'],'splitter':['best','random']}
grid = GridSearchCV( DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 352, in fit
    criterion = CRITERIA_CLF[self.criterion](
KeyError: 'log_loss'

  warnings.warn(some_

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [10]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\base.py:435: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [11]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions )

C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test, grid_predictions,average='weighted')
print("The f1 macro value for parameter {}:".format(grid.best_params_),f1_macro)

The f1 macro value for parameter {'criterion': 'gini', 'max_features': 'sqrt', 'splitter': 'random'}: 0.47029200909249863


In [13]:
print(cm)

[[ 0 51]
 [ 0 82]]


In [14]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.62      1.00      0.76        82

    accuracy                           0.62       133
   macro avg       0.31      0.50      0.38       133
weighted avg       0.38      0.62      0.47       133



In [15]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\base.py:435: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


0.5

In [16]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004325,0.003671,0.002001,0.001675,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.944707,0.903610,0.888286,0.923652,0.906085,0.913268,0.019314,8
1,0.010620,0.006448,0.002200,0.001167,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.963284,0.961755,0.981217,0.943651,0.962264,0.962434,0.011890,1
2,0.004124,0.005468,0.004124,0.005866,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",1.000000,0.883278,0.962264,0.981031,0.962264,0.957767,0.039784,3
3,0.000000,0.000000,0.003126,0.006252,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.926567,0.943699,0.888515,0.943093,0.925146,0.925404,0.020049,7
4,0.003000,0.002449,0.008423,0.005995,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.962963,0.905069,0.906085,0.981217,1.000000,0.951067,0.038946,4
5,0.004727,0.006271,0.000400,0.000799,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",1.000000,0.925272,0.925524,0.981031,0.906085,0.947582,0.036257,6
6,0.003951,0.006053,0.000000,0.000000,entropy,log2,best,"{'criterion': 'entropy', 'max_features': 'log2...",0.962963,0.942166,0.962573,0.943093,0.981031,0.958365,0.014479,2
7,0.008632,0.005209,0.001600,0.001357,entropy,log2,random,"{'criterion': 'entropy', 'max_features': 'log2...",0.945100,0.962264,0.925524,0.925146,0.981217,0.947850,0.021645,5
8,0.004439,0.003007,0.000000,0.000000,log_loss,sqrt,best,"{'criterion': 'log_loss', 'max_features': 'sqr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
9,0.004745,0.004991,0.000000,0.000000,log_loss,sqrt,random,"{'criterion': 'log_loss', 'max_features': 'sqr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
